Imports

In [51]:

import os
import xml.etree.ElementTree as ET
import json
from collections import defaultdict
import lxml.etree as lxml_ET

Helper functions

In [52]:
def build_element_tree(root):
    def recursive_build(element):
        if element.tag != 'Item':
            return None
        
        item_id = element.find('ID').text
        
        children = []
        for child in element.find('Children') or []:
            child_tree = recursive_build(child)
            if child_tree:
                children.append(child_tree)
        
        return {
            'id': item_id,
            'children': children,
            'properties': set()  # Initialize with an empty set
        }

    tree = []
    for item in root.find('.//Items'):
        item_tree = recursive_build(item)
        if item_tree:
            tree.append(item_tree)
    
    return tree

def get_properties(root):
    properties = defaultdict(set)
    for prop_def in root.findall('.//PropertyDefinition'):
        name_elem = prop_def.find('Name')
        if name_elem is not None:
            prop_name = name_elem.text
            for class_id in prop_def.findall('.//ClassificationID/ItemID'):
                if class_id is not None:
                    item_id = class_id.text
                    properties[item_id].add(prop_name)
    return properties

def assign_properties(tree, properties):
    def recursive_assign(node, parent_properties=None):
        # If the node doesn't have properties, assign parent's properties
        if not node['properties'] and parent_properties:
            node['properties'] = set(parent_properties)
        
        # If the node has properties (either initial or inherited), use them
        # Otherwise, use an empty set
        current_properties = node['properties'] if node['properties'] else set()
        
        # Add any properties specific to this node from the original properties dict
        if node['id'] in properties:
            current_properties.update(properties[node['id']])
        
        # Assign the final set of properties to the node
        node['properties'] = current_properties
        
        # Recursively assign properties to children
        for child in node['children']:
            recursive_assign(child, current_properties)

    # Start the recursive assignment for each top-level node
    for node in tree:
        recursive_assign(node)

def sets_to_lists(obj):
    if isinstance(obj, dict):
        return {k: sets_to_lists(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [sets_to_lists(v) for v in obj]
    elif isinstance(obj, set):
        return list(obj)
    else:
        return obj

Export existing config (testing purposes)

In [53]:
def export_config_prev(element_tree):
    def recursive_export(node, parent_properties=None):
        if parent_properties is None:
            parent_properties = set()

        config_node = {
            'id': node['id']
        }

        not_inherited = parent_properties - node['properties']
        if not_inherited and len(node['properties']) > 0:
            config_node['not_inherited_from'] = not_inherited

        new_props = node['properties'] - parent_properties
        if new_props and node['properties'] != parent_properties:
            config_node['new_properties'] = new_props

        children = []
        for child in node['children']:
            child_config = recursive_export(child, node['properties'])
            if child_config:
                children.append(child_config)

        if children:
            config_node['children'] = children

        if node['children']:
            never_inherit = node['properties'] - set().union(*(child['properties'] for child in node['children']))
            if never_inherit:
                config_node['never_inherit_to'] = never_inherit

        return config_node if len(config_node) > 1 else None

    config_prev = []
    for root_node in element_tree:
        root_config = recursive_export(root_node)
        if root_config:
            config_prev.append(root_config)

    return config_prev

def clean_and_convert(obj):
    if isinstance(obj, dict):
        return {k: clean_and_convert(v) for k, v in obj.items() if v}
    elif isinstance(obj, list):
        return [clean_and_convert(v) for v in obj if v]
    elif isinstance(obj, set):
        return list(obj) if obj else None
    else:
        return obj
    
# Function to convert sets to lists for JSON serialization
def sets_to_lists(obj):
    if isinstance(obj, dict):
        return {k: sets_to_lists(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [sets_to_lists(v) for v in obj]
    elif isinstance(obj, set):
        return list(obj)
    else:
        return obj

Process config and apply

In [54]:
def apply_new_config(element_tree, new_config):
    def find_config_node(config, node_id):
        for item in config:
            if item['id'] == node_id:
                return item
            if 'children' in item:
                result = find_config_node(item['children'], node_id)
                if result:
                    return result
        return None

    def process_node(node, parent_properties=None):
        if parent_properties is None:
            parent_properties = set()

        config_node = find_config_node(new_config, node['id'])

        node['properties'] = set(parent_properties)

        if config_node:
            if 'not_inherited_from' in config_node:
                node['properties'] -= set(config_node['not_inherited_from'])
            if 'new_properties' in config_node:
                node['properties'] |= set(config_node['new_properties'])

        for child in node['children']:
            process_node(child, node['properties'])

        if config_node and 'never_inherit_to' in config_node:
            for child in node['children']:
                child['properties'] -= set(config_node['never_inherit_to'])

    for root_node in element_tree:
        process_node(root_node)

Update XML Properties

In [55]:
def update_xml_properties(original_xml_path, updated_element_tree, output_xml_path):
    parser = lxml_ET.XMLParser(remove_blank_text=True)
    tree = lxml_ET.parse(original_xml_path, parser)
    root = tree.getroot()

    # Find the PropertyDefinitionGroups element
    prop_def_groups = root.find('.//PropertyDefinitionGroups')
    if prop_def_groups is None:
        print(f"Error: No PropertyDefinitionGroups found in {original_xml_path}.")
        return

    # Collect all properties and their associated class IDs
    property_classes = {}
    def collect_properties(node):
        for prop in node['properties']:
            if prop not in property_classes:
                property_classes[prop] = set()
            property_classes[prop].add(node['id'])
        for child in node.get('children', []):
            collect_properties(child)

    for root_node in updated_element_tree:
        collect_properties(root_node)

    # Clear existing PropertyDefinitionGroups
    prop_def_groups.clear()

    # Create a new PropertyDefinitionGroup
    new_group = lxml_ET.SubElement(prop_def_groups, 'PropertyDefinitionGroup')
    lxml_ET.SubElement(new_group, 'Name').text = 'Updated Properties'
    lxml_ET.SubElement(new_group, 'Description')
    prop_defs = lxml_ET.SubElement(new_group, 'PropertyDefinitions')

    # Add PropertyDefinitions
    for prop_name, class_ids in property_classes.items():
        new_prop_def = lxml_ET.SubElement(prop_defs, 'PropertyDefinition')
        lxml_ET.SubElement(new_prop_def, 'Name').text = prop_name
        lxml_ET.SubElement(new_prop_def, 'Description')
        value_desc = lxml_ET.SubElement(new_prop_def, 'ValueDescriptor', Type="SingleValueDescriptor")
        lxml_ET.SubElement(value_desc, 'ValueType').text = 'String'
        lxml_ET.SubElement(new_prop_def, 'MeasureType').text = 'Default'
        default_value = lxml_ET.SubElement(new_prop_def, 'DefaultValue')
        lxml_ET.SubElement(default_value, 'DefaultValueType').text = 'Basic'
        variant = lxml_ET.SubElement(default_value, 'Variant', Type="StringVariant")
        lxml_ET.SubElement(variant, 'Status').text = 'UserUndefined'
        class_ids_elem = lxml_ET.SubElement(new_prop_def, 'ClassificationIDs')
        for class_id in class_ids:
            class_id_elem = lxml_ET.SubElement(class_ids_elem, 'ClassificationID')
            lxml_ET.SubElement(class_id_elem, 'ItemID').text = class_id
            lxml_ET.SubElement(class_id_elem, 'SystemIDName').text = 'ARCHICAD Classification'
            lxml_ET.SubElement(class_id_elem, 'SystemIDVersion').text = 'v 2.0'

    tree.write(output_xml_path, encoding='UTF-8', xml_declaration=True, pretty_print=True)
    print(f"Updated XML saved to {output_path}")

Main Execution

In [56]:
input_folder = 'inputs'
output_folder = 'outputs'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Load the new configuration
with open('config.json', 'r') as f:
    new_config = json.load(f)

for filename in os.listdir(input_folder):
    if filename.endswith('.xml'):
        input_path = os.path.join(input_folder, filename)
        output_filename = f"{os.path.splitext(filename)[0]}_processed.xml"
        output_path = os.path.join(output_folder, output_filename)
        
        tree = ET.parse(input_path)
        root = tree.getroot()
        element_tree = build_element_tree(root)
        with open(f"{os.path.splitext(filename)[0]}_initialtree.json", "w") as f:
            json.dump(element_tree, f, indent=2, default=lambda x: list(x) if isinstance(x, set) else x)
        properties = get_properties(root)
        assign_properties(element_tree, properties)
        
        with open(f"{os.path.splitext(filename)[0]}_intermediatetree.json", "w") as f:
            json.dump(element_tree, f, indent=2, default=lambda x: list(x) if isinstance(x, set) else x)
        
        #apply_new_config(element_tree, new_config)
        
        with open(f"{os.path.splitext(filename)[0]}_configuredtree.json", "w") as f:
            json.dump(element_tree, f, indent=2, default=lambda x: list(x) if isinstance(x, set) else x)
        
        update_xml_properties(input_path, element_tree, output_path)

print("All XML files processed.")

Updated XML saved to outputs\input_processed.xml
All XML files processed.


C:\Users\CDRI\AppData\Local\Temp\ipykernel_13052\508154817.py:9: DeprecationWarning: Testing an element's truth value will raise an exception in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  for child in element.find('Children') or []:
